# **BERTopic**

In [ ]:
import warnings
warnings.simplefilter("ignore")

In [ ]:
import pandas as pd
df = pd.read_csv('agr_en_train.csv')
df = df.dropna()
devdf = pd.read_csv('agr_en_train.csv')
devdf = devdf.dropna()
x = df.drop(['source','label'],axis = 1)
devx = devdf.drop(['source','label'],axis = 1)
devy = devdf['label']

#x.columns = [''] * len(x.columns)

y = df['label']
x = x.append(devx)
y = y.append(devy)
print(x.shape,y.shape)
print(type(x))

(23998, 1) (23998,)
<class 'pandas.core.frame.DataFrame'>


<ipython-input-4-4f789fc7f68f>:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  x = x.append(devx)
<ipython-input-4-4f789fc7f68f>:12: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  y = y.append(devy)


In [ ]:
df.tail()

,source,comment,label
11994,facebook_corpus_msr_394638,They belong to you flight dirty terrorist coun...,OAG
11995,facebook_corpus_msr_429177,"Really motivating programme, congratulations t...",NAG
11996,facebook_corpus_msr_2032370,fabricated news,OAG
11997,facebook_corpus_msr_1722926,What's wrong with you secular idiots,OAG
11998,facebook_corpus_msr_327960,Looks like inevitable after all political hard...,NAG


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11999 entries, 0 to 11998
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   source   11999 non-null  object
 1   comment  11999 non-null  object
 2   label    11999 non-null  object
dtypes: object(3)
memory usage: 281.4+ KB


In [ ]:
df.isnull().sum()

source     0
comment    0
label      0
dtype: int64

In [ ]:
import spacy
import spacy as s
from nltk import stem  # porter stemmer
import os
import pickle#storing and retreiving indexes
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer,TfidfTransformer
from sklearn.linear_model import LogisticRegression,LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier,DecisionTreeRegressor
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB,BernoulliNB
import tensorflow as tf
from tensorflow.keras.layers import Embedding,LSTM,Dense,Dropout,SpatialDropout1D,Conv1D,MaxPooling1D,GRU,BatchNormalization
from tensorflow.keras.layers import Input,Bidirectional,GlobalAveragePooling1D,GlobalMaxPooling1D,concatenate,LeakyReLU
from tensorflow.keras.models import Sequential
from tensorflow.keras import regularizers
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import one_hot
import numpy as np
import spacy
import spacy as s
from nltk import stem  # porter stemmer

In [ ]:
import re
def preprocess_text(sen):

    # Remove punctuations and numbers
    sentence = re.sub('[^a-zA-Z]', ' ', sen)

    # Single character removal
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)

    # Removing multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)

    return sentence

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
sw_nltk = stopwords.words('english')
sw_nltk

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

# **NEW**

In [ ]:
pip install bertopic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.4/143.4 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 19.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 kB 11.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 8.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 54.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 58.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 33.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 60.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 89.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

In [ ]:
import pandas as pd
import numpy as np
import spacy
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import tensorflow as tf
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, GlobalAveragePooling1D
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from bertopic import BERTopic

In [ ]:
# Load the data

train_df = pd.read_csv('agr_en_train.csv')
test_df = pd.read_csv('agr_en_test.csv')

In [ ]:
# Preprocessing

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
nlp = spacy.load('en_core_web_sm')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
def preprocess_text(text):
    doc = nlp(text)
    tokens = [token.lemma_ for token in doc if not token.is_punct and not token.is_stop]
    clean_text = ' '.join(tokens)
    return clean_text

train_df['clean_comment'] = train_df['comment'].apply(preprocess_text)
test_df['clean_comment'] = test_df['comment'].apply(preprocess_text)

In [ ]:
# Create TF-IDF vectorizer

vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = vectorizer.fit_transform(train_df['clean_comment'])
X_test_tfidf = vectorizer.transform(test_df['clean_comment'])

In [ ]:
# Convert labels to numerical values

label_mapping = {'OAG': 0, 'CAG': 1, 'NAG': 2}
train_df['label_id'] = train_df['label'].map(label_mapping)
test_df['label_id'] = test_df['label'].map(label_mapping)

In [ ]:
# Split data

X_train, X_val, y_train, y_val = train_test_split(X_train_tfidf, train_df['label_id'], test_size=0.2, random_state=42)

In [ ]:
# Build the deep learning model
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(3, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Train the model

model.fit(X_train, y_train, epochs=5, batch_size=64, validation_data=(X_val, y_val))

Epoch 1/5


InvalidArgumentError: ignored

In [ ]:
# Predict labels for the test set

test_predictions = model.predict_classes(X_test_tfidf)

AttributeError: ignored

In [ ]:
# Add predicted labels to the test DataFrame

test_df['predicted_label_id'] = test_predictions
test_df['predicted_label'] = test_df['predicted_label_id'].map({v: k for k, v in label_mapping.items()})


In [ ]:
# Apply BERTopic

topic_model = BERTopic()
topics, _ = topic_model.fit_transform(test_df['clean_comment'])
test_df['topic'] = topics

In [ ]:
# Save the updated test DataFrame

test_df.to_csv('test_results.csv', index=False)